In [1]:
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.9 MB/s eta 0:00:00
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-y56nh5pt
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-y56nh5pt
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369489 sha256=b02c6b5ccc8c3bc42d7067b115ac6a3fc1e5a680119d65161c5e92fd9a5de634
  Stored in directory: /tmp/pip-ephem-wheel-cache-i1mjgr10/wheels/da/2b/4c/d6691fa9597aac8bb85d2ac13b112deb897d5b50f5ad9a37e4
Successfully built clip


In [2]:
import torch
import torchvision
from torchvision import transforms, datasets
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import numpy as np
import os
import random
from google.colab import drive
import clip
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import DataLoader

In [3]:
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
%cd '/content/gdrive/My Drive/data'

/content/gdrive/My Drive/data


In [5]:
def set_random_seed(seed=28):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

set_random_seed(0)

# Load CLIP model and preprocessing pipeline
device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model, preprocess = clip.load("ViT-B/32", device=device)

100%|███████████████████████████████████████| 338M/338M [00:03<00:00, 99.4MiB/s]


In [12]:
class_names=[]
class OneShotDataset(Dataset):
    def __init__(self, root_dir, transform=None, one_shot=True):
        """
        Args:
            root_dir (str): Directory containing image folders for each class.
            transform (callable, optional): A function/transform to apply to images.
            one_shot (bool): Whether to select only one training sample per class.
        """
        self.root_dir = root_dir
        self.transform = transform
        self.one_shot = one_shot
        self.data = []
        self.classes = []  # List of class names
        self.class_to_idx = {}
        self._prepare_data()

    def _prepare_data(self):
        """Prepare the dataset by iterating through class directories."""
        self.classes = sorted(os.listdir(self.root_dir))  # Get all class folders
        self.class_to_idx = {cls_name: idx for idx, cls_name in enumerate(self.classes)}

        for cls_name in self.classes:
            class_names.append(cls_name)
            class_path = os.path.join(self.root_dir, cls_name)
            images = sorted(os.listdir(class_path))  # All images in the class folder

            if self.one_shot:
                # Randomly select only one image per class
                images = [random.choice(images)]

            for img_name in images:
                self.data.append((os.path.join(class_path, img_name), self.class_to_idx[cls_name]))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, label = self.data[idx]
        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image, label

In [7]:
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Random crop and resize
    transforms.RandomHorizontalFlip(),  # Horizontal flip
    transforms.ColorJitter(brightness=0.1, contrast=0.1), # Random rotation
    preprocess  # Apply CLIP preprocessing
])

# Testing dataset: all images
test_dataset = OneShotDataset(root_dir="/content/gdrive/My Drive/data/test", transform=test_transform, one_shot=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [8]:
class CLIPClassifier(nn.Module):
    def __init__(self, clip_model, num_classes):
        super(CLIPClassifier, self).__init__()
        self.clip_model = clip_model
        self.dropout = nn.Dropout(0.4)  # Drop 20% neurons
        self.fc = nn.Linear(clip_model.visual.output_dim, num_classes)

    def forward(self, x):
        with torch.no_grad():
            features = self.clip_model.encode_image(x)

        features = features.float()
        features = self.dropout(features)  # Apply dropout
        return self.fc(features)

In [9]:
def evaluate_clip_classifier(model, test_loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Accuracy: {accuracy:.2f}%")

In [10]:
num_classes = len(test_dataset.classes)
model = CLIPClassifier(clip_model, num_classes).to(device)
model.load_state_dict(torch.load('OneShotwithCLIP_model_v2.0.pth'))

<ipython-input-10-129d68d3851d>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('OneShotwithCLIP_model_v2.0.pth'))


<All keys matched successfully>

In [11]:
evaluate_clip_classifier(model, test_loader)

Accuracy: 71.00%
